In [1]:
import warnings
warnings.filterwarnings('ignore')

import importlib.util
from sklearn.linear_model import Ridge, RidgeCV, BayesianRidge
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression

from sklearn.model_selection import cross_val_score, RandomizedSearchCV, RepeatedKFold, GridSearchCV

from sklearn import metrics
from sklearn.dummy import DummyRegressor

from sklearn.ensemble import RandomForestRegressor

import xgboost as xg


import numpy as np
import pandas as pd
import pickle
from collections import Counter, defaultdict

import altair as alt
from scipy.stats import skew
from sklearn.feature_selection import SelectFromModel
import joblib

# Set Up

Time columns that need to be converted to integer hour for model:

In [2]:
parse_times = ["MKOPEN", "MKCLOSE", "MKEMHOPEN", "MKEMHCLOSE",
               "MKOPENYEST", "MKCLOSEYEST", "MKOPENTOM",
               "MKCLOSETOM","EPOPEN", "EPCLOSE", "EPEMHOPEN",
               "EPEMHCLOSE", "EPOPENYEST", "EPCLOSEYEST",
               "EPOPENTOM", "EPCLOSETOM", "HSOPEN", "HSCLOSE",
               "HSEMHOPEN", "HSEMHCLOSE", "HSOPENYEST", "HSCLOSEYEST",
               "HSOPENTOM", "HSCLOSETOM", "AKOPEN", "AKCLOSE",
               "AKEMHOPEN", "AKOPENYEST", "AKCLOSEYEST","AKEMHCLOSE",
               "AKOPENTOM", "AKCLOSETOM", "MKPRDDT1", "MKPRDDT2",
               "MKPRDNT1", "MKPRDNT2", "MKFIRET1", "MKFIRET2",
               "EPFIRET1", "EPFIRET2", "HSPRDDT1", "HSFIRET1",
               "HSFIRET2", "HSSHWNT1", "HSSHWNT2", "AKPRDDT1",
               "AKPRDDT2", "AKSHWNT1", "AKSHWNT2"]

# Load posted wait time datasets

In [3]:
# specify the module that needs to be
# imported relative to the path of the
# module
spec = importlib.util.spec_from_file_location("loadTrainTestPostedWaitTimes", "../src/data/loadTrainTestData.py")

# creates a new module based on spec
loadTrainPosted = importlib.util.module_from_spec(spec)

# executes the module in its own namespace
# when a module is imported or reloaded.
spec.loader.exec_module(loadTrainPosted)

X_train, X_test, y_train, y_test = loadTrainPosted.loadTrainTestPostedWaitTimes()

### Convert key data points from date to integer

In [4]:
X_train["MONTHOFYEAR"] = X_train["date"].dt.month.astype("Int8")
X_train["YEAR"] = X_train["date"].dt.year.astype("Int16")
X_train["DAYOFYEAR"] = X_train["date"].dt.dayofyear.astype("Int16")
X_train["HOUROFDAY"] = X_train["datetime"].dt.hour.astype("Int8")

X_test["MONTHOFYEAR"] = X_test["date"].dt.month.astype("Int8")
X_test["YEAR"] = X_test["date"].dt.year.astype("Int16")
X_test["DAYOFYEAR"] = X_test["date"].dt.dayofyear.astype("Int16")
X_test["HOUROFDAY"] = X_test["datetime"].dt.hour.astype("Int8")

### Sort by datetime before imputation (keeping y-values associated)

In [5]:
train = pd.concat([X_train, y_train], axis=1).sort_values(['datetime'])
test = pd.concat([X_test, y_test], axis=1).sort_values(['datetime'])

In [6]:
X_train_impute = train.drop(columns=["POSTED_WAIT"])
y_train = train["POSTED_WAIT"]

X_test_impute = test.drop(columns=["POSTED_WAIT"])
y_test = test["POSTED_WAIT"]

In [7]:
# del train, test

### Many open/close times, parade times, etc. are in HH:MM format. 

Convert to integer hour & fill nulls with 99.

This means that particulate event does not exist for that day. (e.g. Magic Kingdom doesn't have a second parade)

In [8]:
for col in parse_times:
    X_train_impute[col] =  X_train_impute[col].fillna("99")
    X_train_impute[f"{col}_HOUR"] = X_train_impute[col].apply(lambda x: x[:2] if x[0]!=0 else x[:1]).astype(int).astype("Int8")
    X_train_impute.drop(columns = col, inplace=True)

In [9]:
for col in parse_times:
    X_test_impute[col] =  X_test_impute[col].fillna("99")
    X_test_impute[f"{col}_HOUR"] = X_test_impute[col].apply(lambda x: x[:2] if x[0]!=0 else x[:1]).astype(int).astype("Int8")
    X_test_impute.drop(columns = col, inplace=True)

### Data Imputation

First impute by backfilling values. For any remaining nulls, impute the median.

In [10]:
for col in X_train_impute.columns:
    nulls = X_train_impute[col].isnull().sum()
    
    if nulls>0:
        print(col)
        X_train_impute[col].fillna(method ='bfill', inplace=True)
    
        if X_train_impute[col].isnull().sum()>0:
            X_train_impute[col].fillna(X_train_impute[col].median(), inplace=True)

WDWMAXTEMP
WDWMINTEMP
WDWMEANTEMP
inSession
inSession_Enrollment
inSession_wdw
inSession_dlr
inSession_sqrt_WDW
inSession_sqrt_DLR
inSession_California
inSession_DC
inSession_Central_FL
inSession_Drive1_FL
inSession_Drive2_FL
inSession_Drive_CA
inSession_Florida
inSession_Mardi_Gras
inSession_Midwest
inSession_NY_NJ
inSession_NY_NJ_PA
inSession_New_England
inSession_New_Jersey
inSession_Nothwest
INSESSION_PLANES
inSession_SoCal
inSession_Southwest


In [11]:
for col in X_test_impute.columns:
    nulls = X_test_impute[col].isnull().sum()
    
    if nulls>0:
        print(col)
        X_test_impute[col]= X_test_impute[col].fillna(method ='bfill')
        
        if X_test_impute[col].isnull().sum()>0:
            X_test_impute[col].fillna(X_test_impute[col].median(), inplace=True)

WDWMAXTEMP
WDWMINTEMP
WDWMEANTEMP
inSession
inSession_Enrollment
inSession_wdw
inSession_dlr
inSession_sqrt_WDW
inSession_sqrt_DLR
inSession_California
inSession_DC
inSession_Central_FL
inSession_Drive1_FL
inSession_Drive2_FL
inSession_Drive_CA
inSession_Florida
inSession_Mardi_Gras
inSession_Midwest
inSession_NY_NJ
inSession_NY_NJ_PA
inSession_New_England
inSession_New_Jersey
inSession_Nothwest
INSESSION_PLANES
inSession_SoCal
inSession_Southwest


### Drop Datetime columns - no longer needed

In [12]:
X_train_encoded = X_train_impute.drop(columns=['date', 'datetime', 'Unnamed: 0'])
X_test_encoded = X_test_impute.drop(columns=['date', 'datetime', 'Unnamed: 0'])

In [13]:
del X_train_impute, X_test_impute

### Use variance threshold to drop more boolean columns.

These columns are more than 99.9% similar.

In [14]:
X_dtype = X_train_encoded.select_dtypes(include=['bool']).reset_index(drop=True)

var_thr = VarianceThreshold(threshold=0.001)  # Removing both constant and quasi-constant
var_thr.fit(X_dtype)

concol = [column for column in X_dtype.columns
          if column not in X_dtype.columns[var_thr.get_support()]]


del var_thr, X_dtype

if "Weather Type" in concol:
    concol.remove("Weather Type")

print(f"DROPPING BOOL: ", concol)
X_train_encoded.drop(concol, axis=1, inplace=True)
X_test_encoded.drop(concol, axis=1, inplace=True)

DROPPING BOOL:  ['HOLIDAYN_ash|val', 'HOLIDAYN_chv|pas', 'HOLIDAYN_cmd|han', 'HOLIDAYN_col|suk', 'HOLIDAYN_hal|nvd', 'HOLIDAYN_njc|vet', 'MKeventN_dah|emm', 'HSeventN_wdwsotf', "HSFIREN_disney's hollywood studios special july 4th fireworks presentation", "HSFIREN_new year's eve fireworks", 'Wind Speed Quality_a', 'Wind Speed Quality_p', 'Wind Speed Quality_passed gross limits check if element is present', 'Cloud Quality Code_erroneous, data originate from an ncei data source', 'Cloud Determination Code_statistically derived', 'Visibiliy Quality Code_p', 'Visibility Variability Code_variable', 'Temperature Quality Code_suspect, data originate from an ncei data source']


### Data Scaling & Transformation

Log Transform skewed numeric columns & then apply StandardScaler to scale numeric columns.

In [15]:
scaler = RobustScaler()


X_dtype_train = X_train_encoded.select_dtypes(include=[np.number]).reset_index(drop=True)
num_cols = list(X_dtype_train.columns)

X_dtype_test = X_test_encoded.select_dtypes(include=[np.number]).reset_index(drop=True)

In [16]:
skewed_cols = []
for col in X_dtype_train.columns:
    skewness = abs(skew(list(X_dtype_train[col])))
    if skewness > 1:
        print(col)
        X_dtype_train[f"log_{col}"] = np.log(X_dtype_train[col].astype(float).apply(lambda x: x+5))
        X_dtype_test[f"log_{col}"] = np.log(X_dtype_test[col].astype(float).apply(lambda x: x+5))                                         
        skewed_cols.append(col)

X_dtype_train.drop(columns=skewed_cols, inplace=True)
X_dtype_test.drop(columns=skewed_cols, inplace=True)

Ride_duration_min
HOLIDAYPX
HOLIDAYM
WDWMAXTEMP
inSession_SoCal
CapacityLost_HS
CapacityLost_AK
CapacityLostWGT_HS
CapacityLostWGT_AK
MKPRDDAY
MKFIREWK
EPFIREWK
new_case
Wind Angle
Wind Speed
Cloud Height
Visibility Distance (M)
MKOPEN_HOUR
MKCLOSE_HOUR
MKOPENYEST_HOUR
MKCLOSEYEST_HOUR
MKOPENTOM_HOUR
MKCLOSETOM_HOUR
EPOPEN_HOUR
EPCLOSE_HOUR
EPEMHOPEN_HOUR
EPEMHCLOSE_HOUR
EPOPENYEST_HOUR
EPCLOSEYEST_HOUR
EPOPENTOM_HOUR
EPCLOSETOM_HOUR
HSOPEN_HOUR
HSCLOSE_HOUR
HSEMHOPEN_HOUR
HSEMHCLOSE_HOUR
HSOPENYEST_HOUR
HSCLOSEYEST_HOUR
HSOPENTOM_HOUR
HSCLOSETOM_HOUR
AKOPEN_HOUR
AKCLOSE_HOUR
AKOPENYEST_HOUR
AKCLOSEYEST_HOUR
AKOPENTOM_HOUR
AKCLOSETOM_HOUR
MKPRDDT1_HOUR
MKPRDDT2_HOUR
MKFIRET1_HOUR
MKFIRET2_HOUR
EPFIRET1_HOUR
EPFIRET2_HOUR
HSFIRET1_HOUR
HSFIRET2_HOUR
HSSHWNT1_HOUR


In [17]:
X_train_norm = scaler.fit_transform(X_dtype_train)
X_test_norm = scaler.transform(X_dtype_test)

X_train_encoded[num_cols] = X_train_norm
X_test_encoded[num_cols] = X_test_norm

In [19]:
del X_train_norm, X_test_norm, X_dtype_train, X_dtype_test

------

# Modeling

## Dummy Regression - Baseline

Setting baseline metrics to see improvement among other models.

In [21]:
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_train_encoded, y_train)
y_predict_dummy_mean = lm_dummy_mean.predict(X_test_encoded)

In [22]:
## print("DUMMY REGRESSOR - MEAN")

print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_predict_dummy_mean))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_predict_dummy_mean))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_predict_dummy_mean)))
print("R-SQUARED: ", metrics.r2_score(y_test, y_predict_dummy_mean))

Mean Absolute Error (MAE): 162.50162595558666
Mean Squared Error (MSE): 84570.05314900333
Root Mean Squared Error (RMSE): 290.8093071911615
R-SQUARED:  0.0


In [23]:
lm_dummy_median = DummyRegressor(strategy = 'median').fit(X_train_encoded, y_train)
y_predict_dummy_median = lm_dummy_median.predict(X_test_encoded)

In [24]:
print("DUMMY REGRESSOR - MEDIAN")

print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_predict_dummy_median))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_predict_dummy_median))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_predict_dummy_median)))
print("R-SQUARED: ", metrics.r2_score(y_test, y_predict_dummy_median))

DUMMY REGRESSOR - MEDIAN
Mean Absolute Error (MAE): 106.54793720058697
Mean Squared Error (MSE): 91049.30566297466
Root Mean Squared Error (RMSE): 301.7437748537236
R-SQUARED:  -0.07661402911212067


## Ridge Regression Grid Search on Alpha

Testing ridge regression with differing levels of alpha.

In [25]:
linRidge = RidgeCV(alphas=[1e-1, 1, 10], scoring='neg_mean_absolute_error').fit(X_train_encoded, y_train)

In [26]:
predLinRidge = linRidge.predict(X_test_encoded)

In [27]:
print("RIDGE REGRESSION GRID SEARCH: ")
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, predLinRidge))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, predLinRidge))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, predLinRidge)))

RIDGE REGRESSION GRID SEARCH: 
Mean Absolute Error (MAE): 141.45532317265173
Mean Squared Error (MSE): 69920.79966675368
Root Mean Squared Error (RMSE): 264.4254141847067


In [28]:
del linRidge

## XGBoost Regressor

```n_estimators = 100```

In [29]:
xgb_r = xg.XGBRegressor(objective ='reg:squarederror',
                  n_estimators = 100, seed = 123)

In [30]:
# Fitting the model
xgb_r.fit(X_train_encoded, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=123,
             reg_alpha=0, reg_lambda=1, ...)

In [31]:
# Predict the model
pred = xgb_r.predict(X_test_encoded)

In [32]:
print("XGBoost n_estimators = 100")
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))

XGBoost n_estimators = 100
Mean Absolute Error (MAE): 81.52782540698011
Mean Squared Error (MSE): 37576.03206247993
Root Mean Squared Error (RMSE): 193.84538184460297


## Bayesian Ridge Regressor

Testing bayesian ridge regression.

In [33]:
BayReg = BayesianRidge()
BayReg.fit(X_train_encoded, y_train)

BayesianRidge()

In [34]:
pred = BayReg.predict(X_test_encoded)

In [35]:
print("Bayesian Ridge Regression: ")
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))

Bayesian Ridge Regression: 
Mean Absolute Error (MAE): 141.46817354167044
Mean Squared Error (MSE): 69926.25973627747
Root Mean Squared Error (RMSE): 264.43573838699916


## Random Forest Regression

Doing grid search manually because GCP throws TerminatedWorkerError and local throws memory error with GridSearchCV:

In [ ]:
important_features = {}
results = {}
feature_dict = defaultdict(list)
feature_importance = defaultdict(dict)
for n_est in (10, 50, 100):
    for max_depth in [10, 50, 100]:
        print(f"STARTING {n_est}, {max_depth}")
        rfc = RandomForestRegressor(n_estimators=n_est, max_depth=max_depth, random_state=0, n_jobs=-1)
        rfc.fit(X_train_encoded, y_train)
        
        pred = rfc.predict(X_test_encoded)
        
        mae = metrics.mean_absolute_error(y_test, pred)
        mse = metrics.mean_squared_error(y_test, pred)
        rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))
        r2 = metrics.r2_score(y_test, pred)
        
        results[f"{n_est}_{max_depth}"] = {"mae": mae, "mse": mse, "rmse":rmse, "r2":r2}
        
        importances = rfc.feature_importances_
        important_feat = np.argsort(importances)[-50:]
        
        selected_feat_names= list(X_train.columns[important_feat])
        
        for idx, x in enumerate(important_feat):
            feature_dict[selected_feat_names[idx]].append(importances[x])
            
            try:
                feature_importance[selected_feat_names[idx]]["count"] += 1
                feature_importance[selected_feat_names[idx]]["importances"].append(importances[x])
            except KeyError:
                feature_importance[selected_feat_names[idx]]["count"] = 1
                feature_importance[selected_feat_names[idx]]["importances"] = [importances[x]]
            
    
print(results)

STARTING 10, 10
STARTING 10, 50
STARTING 10, 100
STARTING 50, 10
STARTING 50, 50


## RF Grid Search Results

It appears that the sweet spot for metrics optimization vs. model complexity is 10 estimators and max of 50 trees.

In [ ]:
gridSearchResults = pd.DataFrame.from_dict(results).T.reset_index()

gridSearchResults[['n_estimators', 'max_depth']] = gridSearchResults['index'].str.split("_", expand=True).astype(int)
gridSearchMetrics = gridSearchResults.melt(id_vars=["index", "n_estimators", "max_depth"])

metricsChart = alt.Chart(gridSearchMetrics).mark_bar().encode(
    x='max_depth:N',
    y='value:Q',
    color=alt.Color('variable:N', scale = alt.Scale(range=["#EAC2B1", "#90C6FA", "#F8E467", "#2C7AAF"])),
    row='n_estimators:N',
    column='variable:N',
).resolve_scale(
  y='independent'
).properties(
    title={
    "text": ["Performance Metrics by n_estimators & max_depth"],
      "subtitle": ["Random Forest Model"]
    }
)


metricsChart

Top 50 Features sorted on number of Grid Search random forest models that consider it important and mean importance among them:

In [30]:
for x in feature_importance:
    feature_importance[x]["mean_importance"] = np.mean(feature_importance[x]["importances"])

In [31]:
sorted(feature_importance, key=lambda x: (-feature_importance[x]['count'], -feature_importance[x]["mean_importance"]))[:50]

['EPOPENYEST',
 'Age_interest_adults',
 'Age_interest_teens',
 'Height_req_inches',
 'WDWSEASON_none',
 'HSHOURSYEST',
 'EPHOURSYEST',
 'MKOPENTOM',
 'MKFIREN_fantasy in the sky fireworks',
 'AKOPEN',
 'EPEMHOPEN',
 'WDWSEASON_columbus day',
 'MKCLOSEYEST',
 'MKHOURSYEST',
 'MKCLOSETOM',
 'WDWSEASON_memorial day',
 'WDWSEASON_september low',
 'DAYOFYEAR',
 'MKOPENYEST',
 'HOLIDAYN_lab',
 'INSESSION_PLANES',
 'MKFIREN_happily ever after',
 'Age_of_ride_days',
 'MKeventN_mvmcp',
 'HSOPENTOM',
 'Age_of_ride_years',
 'MKeventN_mnsshp',
 'WDWSEASON_thanksgiving',
 'inSession_NY_NJ',
 'WDWeventN_wdwdd|wdwhol',
 'EPCLOSEYEST',
 'Wind Speed Quality_p',
 'Ride_type_slow',
 'MKPRDNN_main street electrical parade',
 'MKEMHCLOSE',
 'Ride_name_dumbo the flying elephant',
 'HSEMHEYEST',
 'inSession',
 'MKEMHMORN',
 'MKPRDNN_mickey\'s "boo-to-you" halloween parade',
 'MKPRDNN_none',
 "MKFIREN_disney's celebrate america! - a fourth of july concert in the sky",
 'HSHOURSTOM',
 'WEEKOFYEAR',
 "MKFIREN_d

----

These parameters seem to have the best balance of simplicity and low error.

##### ```n_estimators=10``` ```max_depth=50```

In [43]:
# create regressor object
# rf = RandomForestRegressor(n_estimators = 10, max_depth=50, n_jobs=-1, random_state = 0)
 
# # fit the regressor with x and y data
# rf.fit(X_train_encoded, y_train)

# joblib.dump(rf, 'random_forest10_50_pkl' + '.gz', compress=('gzip', 4))  

#########
# OR 
#########

# load saved model
rf = joblib.load('random_forest10_50_pkl'  + '.gz')

In [44]:
pred = rf.predict(X_test_encoded)

In [45]:
print("RANDOM FOREST n_estimators = 100 : ") 
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))
print("R-SQUARED: ", metrics.r2_score(y_test, pred))

RANDOM FOREST n_estimators = 100 : 
Mean Absolute Error (MAE): 32.60095335124882
Mean Squared Error (MSE): 14949.679666757667
Root Mean Squared Error (RMSE): 122.26888265931633
R-SQUARED:  0.823227264142569


In [46]:
importances = rf.feature_importances_
#
# Sort the feature importance in descending order
#
sorted_indices = np.argsort(importances)[::-1]

In [47]:
X_train_encoded.columns[sorted_indices[:50]]

Index(['CapacityLostWGT_EP', 'Ride_duration_min', 'Age_of_ride_years',
       'Age_of_ride_days', 'Ride_name_walt disney's carousel of progress',
       'AKHOURSTOM', 'CapacityLostWGT_AK', 'CapacityLost_AK',
       'EPOPENYEST_HOUR', 'EPOPEN_HOUR', 'Cloud Height', 'EPCLOSE_HOUR',
       'WDW_TICKET_SEASON_none', 'Ride_name_prince charming regal carrousel',
       'EPEMHOPEN_HOUR', 'AKHOURS', 'YEAR', 'Weather Type', 'HOLIDAYPX',
       'HSHOURSTOM', 'Park_area_fantasyland',
       'Ride_name_tomorrowland transit authority peoplemover', 'DAYOFWEEK',
       'Visibility Distance (M)', 'MKEMHMORN', 'TA_Stars', 'MKeventN_dah',
       'EPCLOSEYEST_HOUR', 'HSHOURSYEST', 'MKFIREN_happily ever after',
       'MKFIREN_happy hallowishes fireworks', 'EPEMHCLOSE_HOUR',
       'inSession_New_Jersey', 'WEATHER_WDWHIGH', 'Ride_name_jungle cruise',
       'inSession_dlr', 'CapacityLostWGT_HS', 'inSession_Nothwest',
       'Park_area_tomorrowland', 'AKHOURSYEST', 'Height_req_inches',
       'HOLIDAYM', '